In [1]:

print("Begin init")
%run -i init_notebook.ipynb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from structlog import get_logger
from tqdm import tqdm

from src.model.StockModel import StockModel
from src.repository.FileOhlcRepository import FileOhlcRepository
from src.repository.SymbolRepository import SymbolRepository
from src.scrapper.YahooScrapper import YahooScrapper
from datetime import datetime, timedelta
print("Loading modules complete", datetime.now())

Begin init


Loading modules complete 2021-06-03 01:59:25.063694


In [2]:
import os
new_root = os.getcwd().replace("nootebok", "")
os.chdir(new_root)
print('New root', os.getcwd())

New root D:\dev\dixi


In [3]:
from src.data_gen.dataset_generator import get_1min_data
stock = StockModel.parse("NASDAQ:AAPL")
min1_df = get_1min_data(stock)
min1_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 77309 entries, 2020-07-13 14:53:00 to 2021-06-02 19:59:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    77309 non-null  datetime64[ns]
 1   low     77309 non-null  float64       
 2   close   77309 non-null  float64       
 3   open    77309 non-null  float64       
 4   high    77309 non-null  float64       
 5   volume  77309 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 4.1 MB


In [4]:
import tensorflow as tf
original_dataset = tf.data.experimental.load("c:\\_data_for_training\\MSFT.dataset")

In [5]:
def check_balance(df):
    count = 0
    positive = 0

    for x, y in tqdm(df):
        s = tf.math.reduce_sum(y).numpy()
        positive = positive + s
        count = count + y.shape[0]
    print("Y", positive, count, 100*positive/count)
check_balance(original_dataset)

100%|██████████| 151658/151658 [00:24<00:00, 6311.52it/s]

Y 13619 151658 8.980073586622533


In [ ]:
from src.calulation.utils import calculate_scalping_for_long_fast, calculate_scalping_for_short_fast

df = min1_df.copy()
df['profit'] = calculate_scalping_for_long_fast(min1_df, 0.50/100, 0.25/100, 30)
positive_count = len(df[df['profit']==1])
all_count = len(df)
print(positive_count/all_count,  positive_count, all_count)

In [ ]:


from src.calulation.utils import normilize_data_
from src.data_gen.dataset_generator import get_inidicators

from src.calulation.indicator_builder import build_indicatrors

indicators = get_inidicators()


df = min1_df.tail(10000).copy()
basic_price = df.iloc[0].low
basic_volume = df.iloc[0].volume
normilize_data_(df, basic_price, basic_volume)

min1_df_with_indicators = build_indicatrors(df, indicators)
min1_df_with_indicators = min1_df_with_indicators.iloc[7000:-1] # drop warmup

min1_df_with_indicators = min1_df_with_indicators.round(5)
print(min1_df_with_indicators.info())

In [ ]:
import src.calulation.utils as ta_utils
ta_utils.fit_scalers(min1_df_with_indicators)
print("Fitting complete")


In [ ]:
# import src.calulation.utils as ta_utils
# min1_df_with_indicators = ta_utils.scaler_df(min1_df_with_indicators)
# print("Scaling complete")

In [ ]:
# print(min1_df_with_indicators.iloc[-1].to_dict())
raw_columns = {'date', 'low', 'close', 'high', 'open', 'volume'}
for col in min1_df_with_indicators.columns:
    if col in raw_columns:
        continue

    print(f"{col}")
    plt.title(col)
    min1_df_with_indicators[col].hist(bins=100)
    plt.show()
    print(f"Min:{min1_df_with_indicators[col].min()}")
    print(f"Max:{min1_df_with_indicators[col].max()}")
    abs = np.abs(min1_df_with_indicators[col])
    print()
    print("mean", np.mean(abs))
    print("75%", np.percentile(abs, 75))
    print("80%", np.percentile(abs, 80))
    print("90%", np.percentile(abs, 99))
    print("99%", np.percentile(abs, 99))
    print("*"*100)

In [ ]:
d = min1_df_with_indicators.copy()
d.reset_index(inplace=True, drop=True)
d[['tema(n=5)','tema(n=16)', 'tema(n=32)', 'tema(n=64)', 'tema(n=128)', 'tema(n=256)', 'tema(n=512)']].plot()

In [ ]:
# min1_df_with_indicators['profit'], min1_df_with_indicators['loss'] = ta_utils.get_long_profit_and_loss(min1_df_with_indicators, l=15)


In [ ]:
# min1_df_with_indicators['profit'].hist(bins=100)
# plt.show()
#
# min1_df_with_indicators['loss'].hist(bins=100)
# plt.show()

In [ ]:
# positive = min1_df_with_indicators[min1_df_with_indicators['profit']>0.25/100]
# print(100*len(positive)/len(min1_df_with_indicators))
#
# positive = min1_df_with_indicators[min1_df_with_indicators['loss']>0.5/100]
# print(100*len(positive)/len(min1_df_with_indicators))
#
#
# print(100* len(positive) / len(all_y))